In [1]:
import tensorflow as tf

C:\Users\WELCOME\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\WELCOME\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\WELCOME\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\WELCOME\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarnin

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
minst=input_data.read_data_sets("MNIST_data/",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [15]:
input_width=28
input_heights=28
input_channel=1
input_pixels=784

conv1_k = 5
conv2_k = 5
n_conv1=32
n_conv2=64
n_stride1=1
n_stride2=1
n_maxpool1=2
n_maxpool2=2

n_hidden=1024
n_out=10

input_for_hidden_layer=(input_width//(n_maxpool1*n_maxpool2))*(input_heights//(n_maxpool1*n_maxpool2))*n_conv2

In [16]:

weights={
    "wc1":tf.Variable(tf.random_normal([conv1_k,conv1_k,input_channel,n_conv1])),
    "wc2":tf.Variable(tf.random_normal([conv2_k,conv2_k,n_conv1,n_conv2])),
    "whd":tf.Variable(tf.random_normal([input_for_hidden_layer,n_hidden])),
    "wout":tf.Variable(tf.random.normal([n_hidden,n_out]))
}
baises={
    "bc1":tf.Variable(tf.random_normal([n_conv1])),
    "bc2" : tf.Variable(tf.random_normal([n_conv2])),
    "bhd":tf.Variable(tf.random_normal([n_hidden])),
    "bout":tf.Variable(tf.random_normal([n_out]))
}

In [24]:
def conv(x,weight,bais,strides=1):
    out=tf.nn.conv2d(x,weight,padding="SAME",strides=[1,strides,strides,1])
    out=tf.nn.bias_add(out,bais)
    out=tf.nn.relu(out)
    return out

def maxpool(x,k=2):
    return tf.nn.max_pool(x,padding="SAME",ksize=[1,k,k,1],strides=[1,k,k,1])

In [45]:
def cnn(x,weights,baises,keep_prob):
    x=tf.reshape(x,shape=[-1,input_heights,input_width,input_channel])
    conv1=conv(x,weights["wc1"],baises["bc1"],n_stride1)
    maxpool1=maxpool(conv1,n_maxpool1)
    conv2=conv(maxpool1,weights["wc2"],baises["bc2"])
    maxpool2=maxpool(conv2,n_maxpool2)
    
    hidden_layer_input=tf.reshape(maxpool2,shape=[-1,input_for_hidden_layer])
    hidden_layer_output_before_activation_function=tf.add(tf.matmul(hidden_layer_input,weights["whd"]),baises["bhd"])
    hidden_layer_output_before_dropout=tf.nn.relu(hidden_layer_output_before_activation_function)
    hidden_layer_output=tf.nn.dropout(hidden_layer_output_before_dropout,keep_prob)
    
    output=tf.add(tf.matmul(hidden_layer_output,weights["wout"]),baises["bout"])
    return output

In [46]:
x=tf.placeholder("float",[None,input_pixels])
y=tf.placeholder(tf.int32,[None,n_out])
keep_prob=tf.placeholder("float")
pred = cnn(x,weights,baises,keep_prob)

In [47]:
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred,labels=y))


In [48]:
optimizer=tf.train.AdamOptimizer(learning_rate=0.01)
optimize=optimizer.minimize(cost)

In [49]:
sess=tf.Session()
sess.run(tf.global_variables_initializer())

In [52]:
batch_size=100
for i in range(25):
    total_cost=0
    num_batches=int(minst.train.num_examples/batch_size)
    for iter in range(num_batches):
        x_batch,y_batch=minst.train.next_batch(batch_size)
        c,_=sess.run([cost,optimize],feed_dict={x:x_batch,y:y_batch,keep_prob:0.8})
        total_cost+=c
    print(total_cost)


1029996.2509756088
32771.61671133898
19435.482820556164
12014.121395123519
9435.336331776827
8045.840442523798
6220.698225191519
7224.096136886117
6675.008492628112
4778.609671644646
4946.593639261245
4847.943077446311
3562.8756820690573
3663.7879803763235
3901.4972283405
2881.240045113433
4406.315454088312
2922.4862196741933
2253.9355990563818
2183.941158083407
2536.382378892497
2659.16299391782
2307.3906458820393
1835.6235551283883
2082.5658397186685


In [53]:
predictions=tf.argmax(pred,1)
true_label=tf.argmax(y,1)
correct_pred=tf.equal(predictions,true_label)
predictions,true_label,correct_pred=sess.run([predictions,true_label,correct_pred],feed_dict={x:minst.test.images,y:minst.test.labels,keep_prob:1})
predictions,true_label,correct_pred,correct_pred.sum()

(array([7, 2, 1, ..., 4, 5, 6], dtype=int64),
 array([7, 2, 1, ..., 4, 5, 6], dtype=int64),
 array([ True,  True,  True, ...,  True,  True,  True]),
 9848)